In [1]:
# Install necessary libraries if you haven't already
!pip install pandas torch transformers scipy scikit-learn spacy vaderSentiment
!python -m spacy download en_core_web_sm


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 21.1 MB/s  0:00:00m0:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 24.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 49.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 48.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.2/33.2 MB 38.5 MB/s  0:00:00m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.0/875.0 kB 31.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 48.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.9/3.9 MB 45.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.4/11.4 MB 37.3 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 34.1 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29/29 [spacy]m28/29 [spacy]ormers]ub]
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12

## Import Library 

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sys
import os
import re
import spacy
from transformers import pipeline
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer

/home/codespace/.local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_csv('../data/processed/reviews_processed.csv')

In [6]:
current_dir = os.getcwd()
print(f" current dir {current_dir}")

 current dir /workspaces/Fintech-Analytics/notebooks


In [3]:
df.head()

,review_id,review_text,rating,review_date,review_year,review_month,bank_code,bank_name,user_name,thumbs_up,text_length,source
0,3463230e-f9f7-4be3-a632-fdd8d017ce84,🙏👍,5,2025-11-29,2025,11,BOA,Bank of Abyssinia,Yasin Alemu,0,2,Google Play
1,a6cbfa34-f2b1-4a16-96b6-c94f58cea76f,Very Good,5,2025-11-28,2025,11,BOA,Bank of Abyssinia,Wariyo Dida,0,9,Google Play
2,fc67d12c-92e2-45aa-a9e0-011f58a583bc,goof,5,2025-11-28,2025,11,BOA,Bank of Abyssinia,Hailegebrail Tegegn,0,4,Google Play
3,11306fb9-5571-4950-8d32-604c5402242f,good!,5,2025-11-28,2025,11,BOA,Bank of Abyssinia,Tsegay ab,0,5,Google Play
4,809c46d2-730e-446a-9061-2a45e978ad9d,good jop,5,2025-11-27,2025,11,BOA,Bank of Abyssinia,Yohanis Fikadu,0,8,Google Play


In [4]:
# BASIC PREPROCESSING
def clean_text(text):
    if not isinstance(text, str): return ""
    text = text.lower()
    text = re.sub(r'http\S+', '', text) # Remove URLs
    text = re.sub(r'[^a-zA-Z\s]', '', text) # Keep only English letters (adjust if keeping Amharic)
    text = re.sub(r'\s+', ' ', text).strip()
    return text

# Create a cleaned column for Thematic Analysis
df['cleaned_text'] = df['review_text'].apply(clean_text)
print(f"Loaded {len(df)} reviews.")

Loaded 1200 reviews.


In [5]:
# Initialize Models
bert_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
vader_analyzer = SentimentIntensityAnalyzer()

def get_hybrid_sentiment(text):
    # Truncate text to 512 tokens to prevent BERT errors
    text_sample = text[:512] 
            
    # 1. Get VADER scores first (Good for neutrality)
    vader_scores = vader_analyzer.polarity_scores(text_sample)
                        
    # If VADER compound score is clearly Neutral (-0.05 to 0.05), return Neutral
    if -0.05 <= vader_scores['compound'] <= 0.05:
        return "NEUTRAL", 0.0
    
    # 2. If not neutral, use BERT for high-quality binary classification
    try:
        bert_result = bert_pipeline(text_sample)[0]
                                                                    
        label = bert_result['label'] # 'POSITIVE' or 'NEGATIVE'
        score = bert_result['score']
        return label, score
    except:
        return "NEUTRAL", 0.0
        
# Apply to DataFrame (This may take time depending on dataset size
print("Running Sentiment Analysis...")
df[['sentiment_label', 'sentiment_score']] = df['review_text'].apply(lambda x: pd.Series(get_hybrid_sentiment(str(x))))

# --- AGGREGATION ---
print("\nAggregate Sentiment by Bank and Rating:")
agg_df = df.groupby(['bank_name', 'rating'])['sentiment_label'].value_counts(normalize=True).unstack()
print(agg_df.head())

Device set to use cpu


Running Sentiment Analysis...



Aggregate Sentiment by Bank and Rating:
sentiment_label           NEGATIVE   NEUTRAL  POSITIVE
bank_name         rating                              
Bank of Abyssinia 1       0.641791  0.328358  0.029851
                  2       0.400000  0.466667  0.133333
                  3       0.454545  0.227273  0.318182
                  4       0.217391  0.347826  0.434783
                  5       0.067961  0.310680  0.621359


In [6]:
agg_df.tail(10)

sentiment_label                     NEGATIVE   NEUTRAL  POSITIVE
bank_name                   rating                              
Commercial Bank of Ethiopia 1       0.339286  0.571429  0.089286
                            2       0.538462  0.384615  0.076923
                            3       0.434783  0.347826  0.217391
                            4       0.230769  0.333333  0.435897
                            5       0.066914  0.282528  0.650558
Dashen Bank                 1       0.684932  0.246575  0.068493
                            2       0.666667  0.166667  0.166667
                            3       0.318182  0.363636  0.318182
                            4       0.192308  0.269231  0.538462
                            5       0.042146  0.206897  0.750958

In [ ]:
plot_df = agg_df.reset_index()
# Define the order of sentiment categories for consistent stacking and coloring
sentiment_order = ['POSITIVE', 'NEUTRAL', 'NEGATIVE']
# Explicitly using colors for clarity: Green, Yellow, Red
sentiment_colors = ['#4CAF50', '#FFEB3B', '#F44336'] 

# --- Prepare Data for Stacking ---
# Melt the data: This transforms the wide table (one column per sentiment) into a long format 
# suitable for Seaborn's barplot stacking logic (dodge=False).
plot_melt = plot_df.melt(id_vars=['bank_name', 'rating'], 
        value_vars=sentiment_order, 
        var_name='Sentiment',
        value_name='Proportion')

# Ensure the sentiment categories are ordered correctly for stacking (Positive at the bottom)
plot_melt['Sentiment'] = pd.Categorical(plot_melt['Sentiment'], categories=sentiment_order, ordered=True)

# Identify unique banks to create a subplot for each one
banks = plot_df['bank_name'].unique
n_banks = len(banks)

 # --- Create Subplots for Comparison
fig, axes = plt.subplots(n_banks, 1, figsize=(12, 5 * n_banks), sharex=True)
                          
# Handle the case where there is only one bank
                                                                           if n_banks == 1: 
                                                                               axes = [axes]
                                                                                   
                                                                                   for i, bank in enumerate(banks):
                                                                                       bank_data = plot_melt[plot_melt['bank_name'] == bank]
                                                                                           
                                                                                               # Create the stacked bar chart for the current bank
                                                                                                   sns.barplot(
                                                                                                           data=bank_data, 
                                                                                                                   x='rating', 
                                                                                                                           y='Proportion', 
                                                                                                                                   hue='Sentiment', 
                                                                                                                                           palette=sentiment_colors,
                                                                                                                                                   dodge=False, # **Crucial for stacking**
                                                                                                                                                           ax=axes[i],
                                                                                                                                                                   order=sorted(plot_df['rating'].unique())
                                                                                                                                                                       )

                                                                                                                                                                           # Styling and labeling
                                                                                                                                                                               axes[i].set_title(f'Sentiment Distribution by Rating for {bank}', fontsize=14)
                                                                                                                                                                                   axes[i].set_ylabel('Proportion', fontsize=12)
                                                                                                                                                                                       axes[i].set_yticks(np.arange(0, 1.1, 0.2))
                                                                                                                                                                                           axes[i].grid(axis='y', linestyle='--', alpha=0.7)
                                                                                                                                                                                               
                                                                                                                                                                                                   # Only show the legend once, at the bottom
                                                                                                                                                                                                       if i < n_banks - 1:
                                                                                                                                                                                                               axes[i].get_legend().remove()
                                                                                                                                                                                                                   else:
                                                                                                                                                                                                                           axes[i].legend(title='Sentiment', loc='upper center', bbox_to_anchor=(0.5, -0.15), ncol=3)
                                                                                                                                                                                                                                   axes[i].set_xlabel('Review Rating (1-Star to 5-Star)', fontsize=12)


                                                                                                                                                                                                                                   plt.suptitle('Comparative Sentiment Analysis by Bank and Rating', fontsize=16, y=1.02)
                                                                                                                                                                                                                                   plt.tight_layout(rect=[0, 0.05, 1, 1.0])
                                                                                                                                                                                                                                   plt.show() 

IndentationError: unexpected indent (3794859215.py, line 25)